In [1]:
! sudo pip install pysoundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#import the import labraries
import librosa
import os, glob, pickle
import numpy as np
import soundfile
from sklearn.model_selection import train_test_split

In [3]:
#import the dataset and extract features
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
            return result

In [4]:
#define a dictionary to hold numbers and the emotions available in the dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [5]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("kaggle kernels output shivamburnwal/speech-emotion-recognition -p /path/to/dest"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [6]:
# Split the dataset into train and test
x_train,x_test,y_train,y_test=load_data(test_size=12)

ValueError: ignored

In [ ]:
#train.shape
x_train.shape

(576, 180)

In [ ]:
#test.shape
x_test.shape

(192, 180)

In [ ]:
y_train.value_count

In [ ]:
#Bulding MLP classifier model
from sklearn.neural_network import MLPClassifier
model=MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
                    learning_rate='adaptive', max_iter=500)
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
#prediction on test dataset
y_pred=model.predict(x_test)

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix
c=confusion_matrix(y_pred,y_test)
c

array([[29,  7,  2,  5],
       [ 5, 31,  2,  9],
       [ 3,  6, 45,  5],
       [ 0,  3,  5, 35]])

In [ ]:
#calcalate the accurcy
from sklearn.metrics import accuracy_score
z=accuracy_score(y_test,y_pred)
z
print("Accuracy: {:.2f}%".format(z*100))

Accuracy: 72.92%
